In [8]:
## volume calculation


# have a pytorch mesh
from pytorch3d.structures import Meshes
import torch

def get_signed_tet_volume(triangle):
    """
    triangle is a Tensor of 3 x 3, 3 vertices each with 3 coordinates
    """
    v1,v2,v3 = triangle
    return torch.dot(v1,torch.cross(v2,v3))/6.0 # could also move this later

def get_volume(mesh: Meshes):
    volume = 0
    for triangle in mesh.faces_list():
        volume += get_signed_tet_volume(triangle)
    return volume   

def get_parallelepiped_volume(triangle):
    v1,v2,v3 = triangle
    return torch.dot(v1,torch.cross(v2,v3)) # could also move this later

def get_volume(mesh: Meshes):
    volume = 0.0
    for triangle in mesh.faces_list():
        volume += get_parallelepiped_volume(triangle)
    return volume / 6.0

def get_volume_batch(meshes: Meshes):
    faces = meshes.faces_packed()
    faces_to_mesh_idx = meshes.faces_packed_to_mesh_idx()
    mesh_to_first_idx = meshes.mesh_to_faces_packed_first_idx()
    n_meshes = len(mesh_to_first_idx)
    volume = torch.zeros(n_meshes)
    for i in range(n_meshes):
        first_idx = mesh_to_first_idx[i]
        
        while 
    for triangle in mesh.faces_list():
        volume += get_parallelepiped_volume(triangle)
    return volume / 6.0



tensor([0., 0., 1.])


tensor(0.1667)

In [16]:
import torch

def get_signed_tet_volume(face_vertices: torch.Tensor) -> torch.Tensor:
    """
    Compute signed tetrahedron volumes for a batch of faces.

    Args:
        face_vertices (torch.Tensor): Tensor of shape (F, 3, 3), where
                                      F is the number of faces, and each face
                                      consists of 3 vertices in 3D.

    Returns:
        torch.Tensor: A tensor of shape (F,) containing signed volumes.
    """
    v0, v1, v2 = face_vertices[:, 0, :], face_vertices[:, 1, :], face_vertices[:, 2, :]
    
    # Compute determinant of the 3x3 matrix [v0, v1, v2]
    volumes = torch.det(torch.stack([v0, v1, v2], dim=-1)) / 6.0  # Shape: (F,)

    return volumes

def get_volume_batch(meshes: Meshes):
    verts_packed = meshes.verts_packed()  # (sum(V_i), 3)
    faces_packed = meshes.faces_packed()  # (sum(F_i), 3)
    mesh_to_face = meshes.mesh_to_faces_packed_first_idx()  # Index of first face per mesh

    volumes = torch.zeros(meshes.num_meshes(), device=verts_packed.device)

    for i in range(meshes.num_meshes()):
        start = mesh_to_face[i]
        end = start + meshes.num_faces_per_mesh()[i]
        face_vertices = verts_packed[faces_packed[start:end]]  # (F, 3, 3)
        volumes[i] = get_signed_tet_volume(face_vertices).sum()  # Sum over all faces

    return volumes  # Returns a tensor of shape (num_meshes,)


def get_volume_scatter_add(meshes: Meshes):
    verts_packed = meshes.verts_packed()  # (sum(V_i), 3)
    faces_packed = meshes.faces_packed()  # (sum(F_i), 3)
    
    face_vertices = verts_packed[faces_packed]  # (sum(F_i), 3, 3)
    face_volumes = get_signed_tet_volume(face_vertices)  # (sum(F_i),)

    # Sum per mesh using torch_scatter (alternative: torch.segment_sum in newer PyTorch)
    volumes = torch.zeros(meshes.num_meshes(), device=verts_packed.device, dtype=face_volumes.dtype)
    volumes.scatter_add(0, meshes.faces_packed_to_mesh_idx(), face_volumes)

    return volumes  # Shape: (num_meshes,)